# Task 2
## Prediction of m6A sites in all SG-NEx direct RNA-Seq samples 
<ul>
<li> Predict m6A RNA modifications in all samples from the SG-NEx data using our own method </li>
<li> Describe the results and compare them across the different cell lines </li>
<li> Summarise and visualise your observations </li>
</ul>

<b>Submission</b>: Describe the results and compare them across the different cell lines. Summarise and visualise your observations.

# Importing modules and functions

In [15]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import joblib
import numpy as np

In [2]:
%run "../code/data_agg_mean.py"

In [3]:
%run "../code/feature_eng_pipeline.py"

# Loading and processing the dataset 

In [4]:
# A549 Lung cell line 
a549_rep5_df = pd.read_csv("../dataset/CSV/A549_replicate5_data.csv") # change input path accordingly 
a549_rep6_df = pd.read_csv("../dataset/CSV/A549_replicate6_data.csv")

# HCT116 Colon cell line
hct116_rep3_df = pd.read_csv("../dataset/CSV/HCT116_replicate3_data.csv")
hct116_rep3_run4_df = pd.read_csv("../dataset/CSV/HCT116_replicate3_run4_data.csv")
hct116_rep4_run3_df = pd.read_csv("../dataset/CSV/HCT116_replicate4_run3_data.csv")

# HepG2 Liver cell line
hepg2_rep5_df = pd.read_csv("../dataset/CSV/hepG2_replicate5_data.csv")
hepg2_rep6_df = pd.read_csv("../dataset/CSV/hepG2_replicate6_data.csv")

# K562 Leukocytes cell line
k562_rep4_df = pd.read_csv("../dataset/CSV/k562_replicate4_data.csv")
k562_rep5_df = pd.read_csv("../dataset/CSV/k562_replicate5_data.csv")
k562_rep6_df = pd.read_csv("../dataset/CSV/k562_replicate6_data.csv")

# MCF7 Breast cell line
mcf7_rep3_df = pd.read_csv("../dataset/CSV/mcf7_replicate3_data.csv")
mcf7_rep4_df = pd.read_csv("../dataset/CSV/mcf7_replicate4_data.csv")

# List of orig df 
orig_df_list = [a549_rep5_df, a549_rep6_df, hct116_rep3_df, hct116_rep3_run4_df, hct116_rep4_run3_df, hepg2_rep5_df,
                hepg2_rep6_df, k562_rep4_df, k562_rep5_df, k562_rep6_df, mcf7_rep3_df, mcf7_rep4_df ]

Aggregated dataset based on mean 

In [21]:
# A549 Lung cell line
a549_rep5_agg_df = pipeline_nn_unlabelled(data_agg_mean(a549_rep5_df))
a549_rep6_agg_df = pipeline_nn_unlabelled(data_agg_mean(a549_rep6_df))

# HCT116 Colon cell line
hct116_rep3_agg_df = pipeline_nn_unlabelled(data_agg_mean(hct116_rep3_df))
hct116_rep3_run4_agg_df = pipeline_nn_unlabelled(data_agg_mean(hct116_rep3_run4_df))
hct116_rep4_run3_agg_df = pipeline_nn_unlabelled(data_agg_mean(hct116_rep4_run3_df))

# HepG2 Liver cell line
hepg2_rep5_agg_df = pipeline_nn_unlabelled(data_agg_mean(hepg2_rep5_df))
hepg2_rep6_agg_df = pipeline_nn_unlabelled(data_agg_mean(hepg2_rep6_df))

# K562 Leukocytes cell line
k562_rep4_agg_df = pipeline_nn_unlabelled(data_agg_mean(k562_rep4_df))
k562_rep5_agg_df = pipeline_nn_unlabelled(data_agg_mean(k562_rep5_df))
k562_rep6_agg_df = pipeline_nn_unlabelled(data_agg_mean(k562_rep6_df))

# MCF7 Breast cell line
mcf7_rep3_agg_df = pipeline_nn_unlabelled(data_agg_mean(mcf7_rep3_df))
mcf7_rep4_agg_df = pipeline_nn_unlabelled(data_agg_mean(mcf7_rep4_df))

# List of agg df
agg_df_list = [a549_rep5_agg_df, a549_rep6_agg_df, hct116_rep3_agg_df, hct116_rep3_run4_agg_df, hct116_rep4_run3_agg_df, hepg2_rep5_agg_df,
               hepg2_rep6_agg_df, k562_rep4_agg_df, k562_rep5_agg_df, k562_rep6_agg_df, mcf7_rep3_agg_df, mcf7_rep4_agg_df]

Check aggregation 

In [6]:
def check_aggregated(orig_df, agg_df):
    orig_distinct_count = orig_df.groupby(["transcript_name", "json_position", "nucleotide_seq"]).ngroups
    agg_distinct_count = len(agg_df)
    if orig_distinct_count == agg_distinct_count:
        print("Pass")
    else:
        raise Exception(f"Check: Count for {orig_df} not the same as {agg_df}.")

for i in range(12):
    check_aggregated(orig_df_list[i], agg_df_list[i])

Pass
Pass
Pass
Pass
Pass
Pass
Pass
Pass
Pass
Pass
Pass
Pass


# Descriptive analytics 

## 1. Illustrate the approach 
Shows the data processing and analysis approach 

## 2. Relative location of m6A along mRNAs 
Lecture mentioned: Metagene plot to show the relative position of detected m6A sites across all transcripts. m6A expected to occur between the coding sequence and 3' UTR 

## 3. Analyse the features that we are using
Which features differ between modified and unmodified sites? Does this look similar across the cell lines? 

In [ ]:
def probability_df(df, model, threshold, cell_line_name, feature):
    proba = model.predict_proba(df)

    # Update the df with the new label
    pred_labels = (proba[:, 1] >= threshold).astype(int)
    labelled_df = df.copy()
    labelled_df['labels'] = pred_labels
    

In [39]:
def feature_plot(df, model, threshold):
    proba = model.predict_proba(df)

    # Update the df with the new label
    pred_labels = (proba[:, 1] >= threshold).astype(int)
    labelled_df = df.copy()
    labelled_df['labels'] = pred_labels

    return sns.boxplot(x = 'transcript_name', y = 'dwelling_time', hue = 'labels', data = labelled_df)

In [31]:
rfc_model = joblib.load("../random_forest_model.pkl")

In [40]:

# fig, axs = plt.subplots(1, 3, figsize=(12, 6))

feature_plot(a549_rep5_agg_df, rfc_model, 0.7)
# feature_plot(hct116_rep3_agg_df, rfc_model, 0.7)
# feature_plot(hepg2_rep5_agg_df, rfc_model, 0.7, axs[2])

plt.show()

ValueError: Could not interpret value `transcript_name` for `x`. An entry with this name does not appear in `data`.

## 4. Compare m6A across all cell lines 
Do you observe that m6A is similar between cell lines, or different?

Which cell lines are more similar?

How many sites are shared, how many are unique?

## 5. Investigate individual sites/ genes 
Which genes are highly modified?

Which genes show differences across cell lines?

Visualise results for individual genes 